#### Core security practices

Fundamental `layers of security` required for any modern web application:

- strong user validation (hashing),
- stateless authorization (JWT), and
- resilience against abuse (Rate Limiting).

---

Set up the **Security Architecture** for your photo/video application using **FastAPI**, focusing on the two most critical areas: **Authentication (Auth)** and **Rate Limiting**.

## 🔒 1. Project Setup and Dependencies

First, ensure your `uv` environment is ready. We are adding libraries for API security and hashing passwords.

```bash
# Install FastAPI, Uvicorn, password hashing, and authentication dependencies
uv pip install fastapi uvicorn[standard] passlib[bcrypt] python-jose[cryptography] fastapi-limiter
```

-----

## 🔑 2. Secure Authentication (JWT + Hashing)

This is the standard for secure API access. 
We use **JWT (JSON Web Tokens)** for stateless user sessions and **Bcrypt** for secure password hashing.

### A. Core Security Functions (File: `app/security.py`)

This file handles password hashing and token management.

```python
from passlib.context import CryptContext
from datetime import datetime, timedelta, timezone
from jose import jwt

# Configuration
SECRET_KEY = "YOUR_SUPER_SECRET_KEY"  # <-- CHANGE THIS!
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Hashing Context (Use Bcrypt for strong password hashing)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def verify_password(plain_password, hashed_password):
    """Compares a plain password against a hash."""
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password):
    """Generates a Bcrypt hash for a password."""
    return pwd_context.hash(password)

def create_access_token(data: dict):
    """Generates a JSON Web Token (JWT)."""
    to_encode = data.copy()
    expire = datetime.now(timezone.utc) + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    # JWT code: Creates the token by encoding the payload and signing it with the SECRET_KEY
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt
```

### B. User Model and Token Endpoint (File: `app/main.py`)

This demonstrates the login process using FastAPI's dependency injection.

```python
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from typing import Annotated
from .security import verify_password, create_access_token # Assuming you structured files

app = FastAPI()

# 1. Pydantic Model for Token Response
class Token(BaseModel):
    access_token: str
    token_type: str = "bearer"

# 2. Dependency: Defines where the token is expected (Authorization header)
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# **DUMMY USER DATA** (Replace with database lookups later)
DUMMY_USERS = {"user@example.com": {"password_hash": "...", "is_active": True}}

# 3. Login Endpoint (Token generation)
@app.post("/token", response_model=Token)
async def login_for_access_token(form_data: Annotated[OAuth2PasswordRequestForm, Depends]):
    # Note: form_data.username is the email, form_data.password is the plain password
    user_data = DUMMY_USERS.get(form_data.username)

    if not user_data or not verify_password(form_data.password, user_data["password_hash"]):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect username or password",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    # Auth Code: Create the token with the user's identity
    access_token = create_access_token(data={"sub": form_data.username})
    
    return {"access_token": access_token, "token_type": "bearer"}

# 4. Protected Endpoint Example
@app.get("/photos/me")
async def read_current_user_photos(token: Annotated[str, Depends(oauth2_scheme)]):
    """Requires a valid JWT in the Authorization header to access."""
    # Logic to decode and verify the token is handled internally by FastAPI and OAuth2PasswordBearer.
    # In a real app, you would decode the token here to get the user ID.
    return {"message": "Access granted to user photos based on token."}
```

-----

## ⚡ 3. Rate Limiting (DoS Protection)

Rate limiting is crucial for preventing abuse and Denial-of-Service (DoS) attacks. We will use the **`fastapi-limiter`** library.

### A. Initialization (In `app/main.py`)

You must initialize the limiter in the startup event of your FastAPI application.

```python
# Continue in app/main.py

from redis.asyncio import Redis # Redis is required for rate limiting state management
from fastapi_limiter import FastAPILimiter 

# Initialize Redis connection (Redis is the industry standard for fast, shared state)
# NOTE: You MUST have a Redis server running locally or externally for this to work.
redis_client = Redis(host="localhost", port=6379, db=0)

@app.on_event("startup")
async def startup():
    # Initialize the Rate Limiter when the app starts
    await FastAPILimiter.init(redis_client)
    print("Rate Limiter initialized.")

# Optional: Close the limiter on shutdown
@app.on_event("shutdown")
async def shutdown():
    await FastAPILimiter.close()

# B. Applying the Limit to an Endpoint
from fastapi_limiter.depends import RateLimiter

@app.get("/public/search", dependencies=[Depends(RateLimiter(times=5, seconds=10))])
async def public_search():
    """
    Limits this endpoint to 5 requests every 10 seconds per IP address.
    If exceeded, returns HTTP 429 Too Many Requests.
    """
    return {"results": "Search results here."}
```

This compact architecture gives you the fundamental layers of security required for any modern web application: **strong user validation (hashing)**, **stateless authorization (JWT)**, and **resilience against abuse (Rate Limiting)**.